In [13]:
import pandas as pd
import numpy as np
import os
from binance.client import Client
from Binance_api import api_key, secret_key
import time
import plotly.graph_objects as go
import talib as ta
from datetime import datetime
import os
from ta import add_all_ta_features
import warnings
import sklearn
from sklearn.decomposition import PCA
import plotly.express as px
import numpy
import plot_utils 

warnings.filterwarnings('ignore')

In [16]:
from plot_utils import shade_plot


ImportError: cannot import name 'shade_plot' from 'plot_utils' (/opt/homebrew/anaconda3/lib/python3.9/site-packages/plot_utils/__init__.py)

In [3]:
client = Client(api_key, secret_key)

coins = ['BTC', 'ETH']

merge = False
for coin in coins:
    print(f'gathering {coin}...')
    start_str = 'Nov 17, 2022'
    end_str = 'NOV 23, 2022'
    
    klines = client.get_historical_klines(symbol=f'{coin}USDT', interval=client.KLINE_INTERVAL_1MINUTE, start_str=start_str, end_str=end_str)
    
    cols = ['OpenTime',
            f'{coin}-USD_Open',
            f'{coin}-USD_High',
            f'{coin}-USD_Low',
            f'{coin}-USD_Close',
            f'{coin}-USD_Volume',
            'CloseTime',
            f'{coin}-QuoteAssetVolume',
            f'{coin}-NumberOfTrades',
            f'{coin}-TBBAV',
            f'{coin}-TBQAV',
            f'{coin}-ignore']
    
    coin_df = pd.DataFrame(klines,columns=cols)
    
    coin_df['OpenTime'] = [datetime.fromtimestamp(ts / 1000) for ts in coin_df['OpenTime']]
    coin_df['CloseTime'] = [datetime.fromtimestamp(ts / 1000) for ts in coin_df['CloseTime']]
    

    for col in coin_df.columns:
        if not 'Time' in col:
            coin_df[col] = coin_df[col].astype(float)
            
    
    coin_df.to_csv(f'.//data//{coin}_NOV172022_1h.csv')
            

    

gathering BTC...
gathering ETH...


In [4]:
raw_data_path = '/Users/lachlancoles/Documents/bitcoin_bot/data/'

symbols = ['BTC', 'ETH']
filenames = os.listdir(raw_data_path)

normed_cols = ['volume_cmf',
                'volume_mfi',
                'volatility_dcp',
                'trend_psar_down_indicator',
                'trend_psar_up_indicator',
                'trend_stc',
                'momentum_rsi',
                'momentum_stoch_rsi',
                'momentum_stoch_rsi_k',
                'momentum_stoch_rsi_d',
                'momentum_stoch']

if filenames[0] == '.DS_Store':
    filenames.pop(0)
    
hours = 4

Xs = []
ys = []
for file in filenames:
    if file != '.DS_Store':
        df = pd.read_csv(f'{raw_data_path}{file}', parse_dates=['OpenTime', 'CloseTime'])
        symbol = file.split('_')[0]
    
    print(f'preprocessing {symbol}...')
    
    labels = []
    
    dts = []
    for i in range(len(df)):
        a = df['CloseTime']
        dts.append(datetime.strptime(str(df['CloseTime'].iloc[i]).split('.')[0], '%Y-%m-%d %H:%M:%S'))
            
        
    df['CloseTime'] = dts
    
    
    for i in range(len(df)-hours):
        labels.append(df[f'{symbol}-USD_Open'].iloc[i+hours] / df[f'{symbol}-USD_Close'].iloc[i])
        
    df = df.head(len(df)-hours)
    df['label'] = labels
    
    add_all_ta_features(df,
                open=f'{symbol}-USD_Open',
                close=f'{symbol}-USD_Close',
                high=f'{symbol}-USD_High',
                low=f'{symbol}-USD_Low',
                volume=f'{symbol}-USD_Volume',
                        fillna=True)
    
    df= df[50:].reset_index(drop=True)
    df_features = df[normed_cols]
    
    X = (df_features-df_features.min())/(df_features.max()-df_features.min())
    Xs.append(X)
    
    y = df['label']
    ys.append(y)

    #X = pd.concat(Xs)
   # X[['OpenTime', 'CloseTime']] = df[['OpenTime', 'CloseTime']]
   # y = pd.concat(ys)
   # df = df.drop(columns=['Unnamed: 0'])
    


preprocessing BTC...
preprocessing BTC...
preprocessing ETH...
preprocessing ETH...
preprocessing BTC...


In [19]:
df_features

,volume_cmf,volume_mfi,volatility_dcp,trend_psar_down_indicator,trend_psar_up_indicator,trend_stc,momentum_rsi,momentum_stoch_rsi,momentum_stoch_rsi_k,momentum_stoch_rsi_d,momentum_stoch
0,-0.070899,52.641342,0.302761,0.0,0.0,93.267642,42.737917,0.500605,0.803542,0.882832,30.276062
1,-0.068949,52.611741,0.442790,0.0,0.0,96.633821,46.769226,0.700523,0.703716,0.825755,44.278968
2,-0.131842,52.827980,0.434072,0.0,0.0,98.316910,46.549508,0.689626,0.630251,0.712503,43.407192
3,-0.132449,47.461978,0.422448,0.0,0.0,99.158455,46.237602,0.674159,0.688103,0.674023,42.244824
4,-0.108431,41.101140,0.335452,0.0,0.0,99.579228,43.868405,0.556668,0.640151,0.652835,33.545223
...,...,...,...,...,...,...,...,...,...,...,...
10022,-0.097417,28.163410,0.038714,0.0,0.0,3.170405,39.160627,0.001117,0.085519,0.076864,3.871361
10023,-0.091688,26.871562,0.072483,0.0,0.0,1.585202,39.311522,0.007790,0.028188,0.066285,7.248302
10024,-0.043016,30.683530,0.067611,0.0,0.0,0.792601,39.167673,0.001429,0.003445,0.039051,6.761146
10025,0.035078,37.364722,0.400059,0.0,0.0,4.480538,52.059860,0.571506,0.193575,0.075069,40.005905


In [135]:
df

,volume_cmf,volume_mfi,volatility_dcp,trend_psar_down_indicator,trend_psar_up_indicator,trend_stc,momentum_rsi,momentum_stoch_rsi,momentum_stoch_rsi_k,momentum_stoch_rsi_d,momentum_stoch
50,-0.070899,52.641342,0.302761,0.0,0.0,93.267642,42.737917,0.500605,0.803542,0.882832,30.276062
51,-0.068949,52.611741,0.442790,0.0,0.0,96.633821,46.769226,0.700523,0.703716,0.825755,44.278968
52,-0.131842,52.827980,0.434072,0.0,0.0,98.316910,46.549508,0.689626,0.630251,0.712503,43.407192
53,-0.132449,47.461978,0.422448,0.0,0.0,99.158455,46.237602,0.674159,0.688103,0.674023,42.244824
54,-0.108431,41.101140,0.335452,0.0,0.0,99.579228,43.868405,0.556668,0.640151,0.652835,33.545223
...,...,...,...,...,...,...,...,...,...,...,...
10072,-0.097417,28.163410,0.038714,0.0,0.0,3.170405,39.160627,0.001117,0.085519,0.076864,3.871361
10073,-0.091688,26.871562,0.072483,0.0,0.0,1.585202,39.311522,0.007790,0.028188,0.066285,7.248302
10074,-0.043016,30.683530,0.067611,0.0,0.0,0.792601,39.167673,0.001429,0.003445,0.039051,6.761146
10075,0.035078,37.364722,0.400059,0.0,0.0,4.480538,52.059860,0.571506,0.193575,0.075069,40.005905


In [7]:
pca = PCA(n_components=2)
pca.fit(X)
pca.transform(X)
pcas = pd.DataFrame(pca.transform(X),columns=['pca1','pca2'])
pcas['label'] = list(y)

In [172]:
pcas

,pca1,pca2,label
0,0.354928,-0.230798,1.000392
1,0.489055,-0.148637,0.999649
2,0.408554,-0.174888,0.998979
3,0.398996,-0.165139,0.997143
4,0.258278,-0.245277,0.997853
...,...,...,...
10022,-1.131189,-0.075995,1.001387
10023,-1.143156,-0.070086,1.001712
10024,-1.162633,-0.089133,1.001213
10025,-0.566847,0.263562,1.000252


In [8]:
px.scatter(pcas,
         x='pca1',
         y='pca2',
         color='label',
         color_continuous_scale='RdBu',
         range_color=[0.98,1.02],
         color_continuous_midpoint=1)

In [11]:
shade_plot(pcas,'pca1','pca2','label',50)

NameError: name 'shade_plot' is not defined

: 

: 

: 

: 

: 

In [ ]:
all_coins_df.to_csv('BTC_ETH_LTC_16NOV.csv',index=False)

: 

In [ ]:
fig


: 

: 